In [17]:
import pandas as pd
#import datetime as dt

#1
table = pd.read_csv("owid-covid-data.csv")
del table["new_cases_smoothed"]
del table["new_deaths_smoothed"]
del table["new_cases_per_million"]
del table["total_cases_per_million"]

# Suggestion - we can delete these columns in one line of code 
table.drop(columns=['new_cases_smoothed', 'new_deaths_smoothed', 'new_cases_per_million', 'total_cases_per_million'], inplace=True)

#2
table = table.rename(columns={'date': 'Date', 'location': 'Country', 'continent': 'Continent', 'iso_code' : 'ISO_code'})
#print(table)

# Suggestion - we can use inplace=True to avoid re-assigning to the variable
table.rename(columns={'date': 'Date', 'location': 'Country', 'continent': 'Continent', 'iso_code' : 'ISO_code'}, inplace=True)

#3
continent_unique = table['Continent'].unique()
#print(continent_unique)

#4
df2 = table.dropna(subset = ['Continent'])
#print(df2)

# Suggestion - this means 'keep only the rows without null values in continent column'
df2 = table[~table['Continent'].isnull()]


#5
df_temp = df2[['Date', 'Country', 'total_cases', 'total_deaths', 'total_vaccinations']].copy()
df3 = df_temp.groupby(by=['Date', 'Country'], dropna = False).sum()
#print(df3)
#print(df2[['Date', 'Country', 'total_cases', 'total_deaths', 'total_vaccinations']])

# Suggestion - can do this in one line of code and the resulting df has 5 columns 
df3 = df2.groupby(['Date','Country'])[['Date','Country','total_cases','total_deaths','total_vaccinations']].sum().reset_index()


#6
#But it automatically changes Nan to 0 (the prints above can show it)
df3['total_cases'].replace("NaN", 0, inplace=True)
#print(df3)

# Suggestion - more specifically, for replacing nulls, use fillna
df3['total_cases'].fillna(0, inplace=True)

#7
df3['data_column'] = df3.index.get_level_values('Date')
df3['year'] = pd.DatetimeIndex(df3['data_column']).year

df3.loc[df3['year'] == 2020, 'year2020'] = "Year 1"
df3.loc[df3['year'] > 2020, 'year2020'] = "Not year 1"
#print(df3)

# Suggestion - Can do this in one line of code using np.where

df3['year'] = np.where(df3['Date'] < '2021-01-01', 'Year 1', 'Not Year 1')

#8
print("The total number of vaccinations is", df3.at[('2021-03-21','Mexico'), 'total_vaccinations'])

dfdf = df2.loc[df2['Country'].isin(['Uzbekistan'])] 
print("Uzbekistan registered it’s first death on", dfdf['Date'].iloc[0])

print("There are", pd.to_datetime(df2['Date'].iloc[-1]) - pd.to_datetime(df2['Date'].iloc[0]))

#How many countries have had more than 1000000 deaths?

df3.loc[df3['total_deaths'] > 1000000, 'more than mln'] = "Yes"
df3 = df3.dropna(subset = ['more than mln'])
#print(df3)
print(len(df3), "countries have had more than 1000000 deaths")


#Find the top 5 countries with the highest number of total cases
df4 = df_temp.groupby(by= 'Country', dropna = False).sum()
df4 = df4.sort_values(by=['total_deaths'])
print(df4)


The total number of vaccinations is 5612291.0
Uzbekistan registered it’s first death on 2020-03-15
There are 1023 days 00:00:00
219 countries have had more than 1000000 deaths
                            total_cases  total_deaths  total_vaccinations
Country                                                                  
Tuvalu                     8.167100e+04           0.0        1.296690e+05
Jersey                     0.000000e+00           0.0        2.250469e+07
Scotland                   0.000000e+00           0.0        5.106149e+09
Turkmenistan               0.000000e+00           0.0        2.070575e+07
Sint Maarten (Dutch part)  0.000000e+00           0.0        2.546995e+06
...                                 ...           ...                 ...
Russia                     8.715772e+09   187717036.0        4.570942e+10
Mexico                     3.312971e+09   212535705.0        3.378069e+10
India                      2.528836e+10   319278233.0        8.332251e+11
Brazil    